In [1]:
import nltk
import pandas as pd
import ast
import re
import numpy as np

import string 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')

import unidecode
import nltk.corpus
from gensim.models import Word2Vec

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import config
import torch
from transformers import BertTokenizer, BertModel

[nltk_data] Error loading wordnet: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>
/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("/Users/ved/Desktop/Capstone/Dataset/Cleaned_Indian_Food_Dataset.csv")

In [3]:
df.rename(columns = {'Cleaned-Ingredients':'ingredients'}, inplace = True)
for i in range(len(df["ingredients"])):
    df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
    df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
    df["ingredients"][i] = df["ingredients"][i].split(',')

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_38336/1364071601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_38336/1364071601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_38336/1364071601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [4]:
df

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"[salt, amchur , karela , red chilli powder, gr...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"[tomato, salt, chickpea lentils, green chilli,...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"[salt, rice vermicelli noodles , asafoetida , ...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"[tomato, salt, ginger, sorrel leaves , fennel ...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"[tomato, salt, ginger, red chillies, curry, as...",https://www.archanaskitchen.com/images/archana...,12
...,...,...,...,...,...,...,...,...,...
5933,Saffron Paneer Peda Recipe,"2 teaspoons Rose water,1/2 teaspoon Ghee,2 cup...",20,Indian,To begin making the Saffron Paneer Peda recipe...,https://www.archanaskitchen.com/saffron-paneer...,"[saffron strands generous, ghee, paneer, rose ...",https://www.archanaskitchen.com/images/archana...,7
5934,Italian Arancini Rice Balls Recipe With Delici...,"4 sprig Basil leaves - finely chopped,2 cups W...",100,Italian Recipes,To begin making the Italian Arancini Rice Ball...,https://www.archanaskitchen.com/italian-aranci...,"[salt, gorgonzola cheese cubes, wheat bread cr...",https://www.archanaskitchen.com/images/archana...,14
5935,Quinoa Phirnee Recipe (Quinoa Milk Pudding),"3/4 cup Sugar,2 cup Milk - vegans can substitu...",35,Indian,"To begin making Quinoa Phirnee Recipe, place a...",https://www.archanaskitchen.com/quinoa-phirnee...,"[ghee, milk, saffron strands, sugar, cardamom ...",https://www.archanaskitchen.com/images/archana...,8
5936,Ullikadala Pulusu Recipe | Spring Onion Curry,"1 pinch Turmeric powder (Haldi),1/2 teaspoon M...",15,Andhra,To begin making Ullikadala Pulusu Recipe | Spr...,https://www.archanaskitchen.com/ullikadala-pul...,"[salt, tamarind water, mustard seeds, asafoeti...",https://www.archanaskitchen.com/images/archana...,13


In [5]:
print(df["ingredients"][0])

['salt', 'amchur ', 'karela ', 'red chilli powder', 'gram flour ', 'onion', 'cumin seeds ', 'coriander powder', 'turmeric powder', 'sunflower oil']


In [6]:
def ingredient_parser(ingredients):
    # measures and common words (already lemmatized)   
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', ...]
    words_to_remove = ['fresh', 'a', 'red', 'bunch', ...]
    # Turn ingredient list from string into a list 
    if isinstance(ingredients, list):
       pass
    else:
       ingredients = ast.literal_eval(ingredients)
    # We first get rid of all the punctuation
    remove_punctuations = str.maketrans('', '', string.punctuation)
    # initialize nltk's lemmatizer    
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for each_item in ingredients:
        each_item.translate(remove_punctuations)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', each_item)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items]
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # get rid of stop words
        stop_words = set(nltk.corpus.stopwords.words('english'))
        items = [word for word in items if word not in stop_words]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measures]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
           ingred_list.append(' '.join(items))
    return ingred_list

In [7]:
df['parsed'] = df.ingredients.apply(ingredient_parser)

In [8]:
def ingredient_parser_final(ingredient):
    """
    neaten the ingredients being outputted
    """
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ast.literal_eval(ingredient)

    ingredients = ",".join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients
def title_parser(title):
    title = unidecode.unidecode(title)
    return title

In [9]:
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"  # You can use other BERT variants as well
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


# You can use these embeddings for various NLP tasks or similarity calculations

In [10]:
def get_bert_embeddings(tokens):
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    with torch.no_grad():
        outputs = model(torch.tensor([input_ids]))
        embeddings = outputs.last_hidden_state[0]  # Get embeddings for the tokens
    return embeddings

# Apply the function to your DataFrame's tokenized column
df['bert_embeddings'] = df['parsed'].apply(get_bert_embeddings)



In [11]:
def get_recommendations(N, scores):
    """
    Rank scores and output a pandas data frame containing all the details of the top N recipes.
    :param scores: list of cosine similarities
    """
    # load in recipe dataset
    df_recipes = df.copy()
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns=["TranslatedRecipeName", "ingredients", "URL","score","cuisine"])
    count = 0
    for i in top:
        recommendation.at[count, "TranslatedRecipeName"] = title_parser(df_recipes["TranslatedRecipeName"][i])
        recommendation.at[count, "ingredients"] = ingredient_parser_final(
            df_recipes["ingredients"][i]
        )
        recommendation.at[count, "URL"] = df_recipes["URL"][i]
        recommendation.at[count, "cuisine"] = df_recipes["Cuisine"][i]
        recommendation.at[count, "score"] = f"{scores[i]}"
        count += 1
    return recommendation

def get_recs(ingredients, N=5, mean=False):
    """
    Get the top N recipe recomendations.
    :param ingredients: comma seperated string listing ingredients
    :param N: number of recommendations
    :param mean: False if using tfidf weighted embeddings, True if using simple mean
    

    data=df.copy()
    # parse ingredients
    data["parsed"] = data.ingredients.apply(ingredient_parser)
    # create corpus
    corpus = get_and_sort_corpus(data)

    if mean:
        # get average embdeddings for each document
        mean_vec_tr = MeanEmbeddingVectorizer(model)
        doc_vec = mean_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(corpus)
    else:
        # use TF-IDF as weights for each word embedding
        tfidf_vec_tr = TfidfEmbeddingVectorizer(model)
        tfidf_vec_tr.fit(corpus)
        doc_vec = tfidf_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(corpus)
    """

    # create embeddings for input text
    input = ingredients
    # create tokens with elements
    input = input.split(",")
    # parse ingredient list
    input = ingredient_parser(input)
    # get embeddings for ingredient doc
    input = get_bert_embeddings(input)
    # get cosine similarity between input embedding and all the document embeddings
    cos_sim = map(lambda x: cosine_similarity(input, x)[0][0], df['bert_embeddings'])
    scores = list(cos_sim)
    # Filter top N recommendations
    recommendations = get_recommendations(N, scores)
    return recommendations

In [31]:
if __name__ == "__main__":
    # test
    input = "egg, tomato, onion"
    #input  = "eggs, oil, salt, tomato, onion, red chilli powder, turmeric"
    rec = get_recs(input)
    print(rec)

                                TranslatedRecipeName  \
0                       Tangy Tomato Pie Tart Recipe   
1  Savoury Crepes with Za'atar Spiced Chicken Recipe   
2               Saffron Cardamom Creme Brulee Recipe   
3     German Pancakes with Caramelised Apples Recipe   
4                 Andhra Style Peanut Chutney Recipe   

                                         ingredients  \
0  tomato,cheese,salt,mixed herbs ,red chilli fla...   
1  tomato,salt,wheat flour,red bell pepper ,dijon...   
2  double,eggs,cardamom  podsseeds,saffron strand...   
3  eggs,nutmeg powder,apples,milk,sugar,dry ginge...   
4  tomato,salt,rye,cloves garlic,peanuts,curry le...   

                                                 URL               score  \
0  https://www.archanaskitchen.com/tangy-tomato-p...  0.7826375961303711   
1  https://www.archanaskitchen.com/savoury-crepes...  0.7462583780288696   
2  https://www.archanaskitchen.com/saffron-cardam...  0.7437824010848999   
3  https://www.archana

In [20]:
test = pd.read_csv("/Users/ved/Desktop/Capstone/accuracytest_-_Sheet1_1.csv")
actual_recipes = list(test["TranslatedRecipeName"])

In [34]:
len(rec)

5

In [44]:
precision_at_K = 0.1

In [45]:
K = 10# mention it as actual no. of recipies
# Calculate P@K
correct_recommendations=[]
for recipe in actual_recipes:
    for i in rec['TranslatedRecipeName'][:K]:
        if recipe == i:
            correct_recommendations.append(True)
#correct_recommendations = [recipe in rec['TranslatedRecipeName'][:K] for recipe in actual_recipes]
precision_at_K = sum(correct_recommendations) / K

print(f"Precision at {K} of Bert model using BERT embeddings : {precision_at_K}")

Precision at 10 of Bert model using BERT embeddings : 0.1


In [15]:
rec['ingredients'][0]

'cheese,britannia,cheese ,butter,bourbon biscuits other biscuits'